# ASSIGNMENT - 2 - SAIKIRAN PULLABHATLA - A8MOH5

In [1]:
import open3d as o3d
from nuscenes import NuScenes
from nuscenes.utils.data_classes import LidarPointCloud
from pyquaternion import Quaternion
import numpy as np
from nuscenes.utils.geometry_utils import points_in_box
from PIL import Image


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
# Initialize NuScenes
nusc = NuScenes(version='v1.0-mini', dataroot='C:/Users/saiki/3D Sensing and Sensor Fusion/Assignemnt - 2/v1.0-mini', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 1.069 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [5]:

# Function to apply rotation and translation transformations
def apply_transformation(points, trans_vector, rot_quaternion):
    """
    Transform a set of 3D points from one coordinate frame to another.
    Args:
        points: Array of points (N, 3) in the original coordinate system.
        trans_vector: Translation vector (3,).
        rot_quaternion: Rotation quaternion.
    Returns:
        Transformed points in the new coordinate frame.
    """
   
     # Apply transpose and rotation and translation
    transformed = rot_quaternion.rotation_matrix @ points.T 
    transformed += trans_vector.reshape(3, 1)  
    return transformed.T  


# Function to filter out points belonging to moving objects
def remove_moving_objects(points, nusc, lidar_data):
    """
    Filter LiDAR points that belong to dynamic objects like vehicles, bicycles, etc.
    Args:
        points: Array of points (N, 3) in global coordinates.
        nusc: NuScenes dataset instance.
        lidar_data: Lidar sample data dictionary.
    Returns:
        Static points (points not belonging to dynamic objects).
    """
     # Initialize mask: all points are initially "static"
    mask = np.ones(points.shape[0], dtype=bool) 
    
    # Retrieve annotations for the sample
    sample_data = nusc.get('sample', lidar_data['sample_token'])  
    for ann_token in sample_data['anns']:
        annotation = nusc.get('sample_annotation', ann_token)
        
        # Check if annotation corresponds to dynamic objects 
        if annotation['category_name'].startswith(('vehicle', 'bicycle', 'motorcycle')):
            
            # Get the bounding box of the object
            box = nusc.get_box(annotation['token']) 
            
            # Exclude points inside the bounding box
            mask &= ~points_in_box(box, points.T)  
    return points[mask]  



# Function to colorize points based on camera images
def colorize_points(nusc, global_points, ego_pose, lidar_data):
    """
    Project LiDAR points onto camera images and assign RGB values to them.
    Args:
        nusc: NuScenes dataset instance.
        global_points: Array of points (N, 3) in global coordinates.
        ego_pose: Vehicle's ego pose data.
        lidar_data: Lidar sample data dictionary.
    Returns:
        Colors: RGB values for each point.
        Mask: Boolean mask indicating which points have valid color values.
    """
    cameras = ['CAM_FRONT', 'CAM_FRONT_RIGHT', 'CAM_BACK_RIGHT', 'CAM_BACK', 'CAM_BACK_LEFT', 'CAM_FRONT_LEFT']
    colors = np.zeros((global_points.shape[0], 3))  # Initialize color array
    mask = np.zeros(global_points.shape[0], dtype=bool)  # Initialize mask: all points are uncolored initially

    for cam in cameras:  # Loop through all six cameras
        try:
            # Load the camera image and calibration data
            cam_data = nusc.get('sample_data', nusc.get('sample', lidar_data['sample_token'])['data'][cam])
            image = np.array(Image.open(nusc.get_sample_data_path(cam_data['token'])))
            cam_calib = nusc.get('calibrated_sensor', cam_data['calibrated_sensor_token'])

            # Camera extrinsic parameters
            cam_rot = Quaternion(cam_calib['rotation'])
            cam_trans = np.array(cam_calib['translation'])

            # Vehicle pose
            ego_rot = Quaternion(ego_pose['rotation'])
            ego_trans = np.array(ego_pose['translation'])

            # Transform points from global frame to camera frame
            cam_global_rot = ego_rot * cam_rot
            cam_global_trans = ego_trans + ego_rot.rotation_matrix @ cam_trans
            points_cam = (global_points - cam_global_trans) @ cam_global_rot.rotation_matrix

            # Project points onto the camera image
            intrinsic = np.array(cam_calib['camera_intrinsic'])
            proj_points = (intrinsic @ points_cam.T).T
            depths = points_cam[:, 2]

            # Mask points that are in front of the camera
            front_mask = depths > 0
            proj_points = proj_points[front_mask]
            proj_points[:, :2] /= proj_points[:, 2:3]

            # Mask points that fall inside the image dimensions
            in_image = (proj_points[:, 0] >= 0) & (proj_points[:, 0] < image.shape[1]) & \
                       (proj_points[:, 1] >= 0) & (proj_points[:, 1] < image.shape[0])

            for idx, proj in zip(np.where(front_mask)[0][in_image], proj_points[in_image]):
                x, y = int(proj[0]), int(proj[1])
                colors[idx] = image[y, x] / 255.0  # Assign RGB color to the point
                mask[idx] = True
        except:
            continue
    return colors, mask


# STEP 1: Aggregated Point Clouds

In [6]:
scene = nusc.scene[0]
lidar_token = nusc.get('sample', scene['first_sample_token'])['data']['LIDAR_TOP']

clouds_raw = []
while lidar_token:
    # Load LiDAR data
    lidar_data = nusc.get('sample_data', lidar_token)
    lidar_points = LidarPointCloud.from_file(nusc.get_sample_data_path(lidar_token)).points[:3, :].T

    # Transform LiDAR points to the global coordinate system
    calib = nusc.get('calibrated_sensor', lidar_data['calibrated_sensor_token'])
    ego_pose = nusc.get('ego_pose', lidar_data['ego_pose_token'])
    vehicle_points = apply_transformation(lidar_points, np.array(calib['translation']), Quaternion(calib['rotation']))
    global_points = apply_transformation(vehicle_points, np.array(ego_pose['translation']), Quaternion(ego_pose['rotation']))

    # Create and store the point cloud
    cloud = o3d.geometry.PointCloud()
    cloud.points = o3d.utility.Vector3dVector(global_points)
    clouds_raw.append(cloud)
    lidar_token = lidar_data['next']

print("Visualizing Aggregated Point Clouds...")
o3d.visualization.draw_geometries(clouds_raw, window_name="Aggregated Point Clouds")

print("Visualization Aggregated Point Clouds done")


Visualizing Aggregated Point Clouds...
Visualization Aggregated Point Clouds done


# Step 2: Moving Object Filtering

In [7]:
filtered_clouds = []
for cloud in clouds_raw:
    static_points = remove_moving_objects(np.asarray(cloud.points), nusc, lidar_data)
    filtered_cloud = o3d.geometry.PointCloud()
    filtered_cloud.points = o3d.utility.Vector3dVector(static_points)
    filtered_cloud.paint_uniform_color([1, 0, 0])  # Red for static points
    filtered_clouds.append(filtered_cloud)

print("Visualizing Moving Object Filtered Point Clouds...")
o3d.visualization.draw_geometries(filtered_clouds, window_name="Moving Object Filtered Point Clouds")

print("Visualization Moving Object Filtered Point Clouds is done")

Visualizing Moving Object Filtered Point Clouds...
Visualization Moving Object Filtered Point Clouds is done


# Step 3: Colorization of Point Clouds

In [8]:

colorized_clouds = []
lidar_token = nusc.get('sample', scene['first_sample_token'])['data']['LIDAR_TOP']

for cloud in clouds_raw:
    lidar_data = nusc.get('sample_data', lidar_token)
    ego_pose = nusc.get('ego_pose', lidar_data['ego_pose_token'])
    colors, mask = colorize_points(nusc, np.asarray(cloud.points), ego_pose, lidar_data)

    # Apply colors to valid points
    colorized_cloud = o3d.geometry.PointCloud()
    colorized_cloud.points = o3d.utility.Vector3dVector(np.asarray(cloud.points)[mask])
    colorized_cloud.colors = o3d.utility.Vector3dVector(colors[mask])
    colorized_clouds.append(colorized_cloud)
    lidar_token = lidar_data['next']

print("Visualizing Colorized Point Clouds...")
o3d.visualization.draw_geometries(colorized_clouds, window_name="Colorized Point Clouds")

print("Visualization Colorized Point Clouds is done")

Visualizing Colorized Point Clouds...
Visualization Colorized Point Clouds is done
